In [ ]:
import helpers.i2c_gui2_helpers as helpers
import datetime
import numpy as np
import time
import pandas
import sqlite3

##############
from pathlib import Path
from scripts.log_action import log_action_v2

# Set defaults

In [ ]:
chip_names = ["ET2p03_Bare19", "ET2p03_Bare20", "ET2p03_Bare21", "ET2p03_Bare22"]
chip_fignames = ["ETROC 2.03 Bare Board 19", "ETROC 2.03 Bare Board 20", "ETROC 2.03 Bare Board 21", "ETROC 2.03 Bare Board 22"]
chip_figtitles = chip_names

# 'The port name the USB-ISS module is connected to. Default: /dev/ttyACM0'
port = "/dev/ttyACM0"
# I2C addresses for the pixel block and WS
chip_addresses = [0x60, 0x61, 0x62, 0x63]
ws_addresses = [0x40, 0x41, 0x42, 0x43]

fig_outdir = Path('../ETROC-figures/NW_SEU_May2025')
fig_outdir = fig_outdir / 'FNAL'
fig_outdir.mkdir(exist_ok=True, parents=True)

config_outdir = Path('../ETROC-Data/NW_SEU_May2025')
config_outdir = config_outdir / "ChipConfig_FNAL"
config_outdir.mkdir(exist_ok=True, parents=True)

histdir = Path('../ETROC-History/NW_SEU_May2025')
histdir.mkdir(exist_ok=True, parents=True)

log_path = Path('../ETROC-History/NW_SEU_May2025')

out_dir = Path('/home/daq/ETROC2/ETROC-Data/NW_SEU_May2025')
out_dir.mkdir(exist_ok=True, parents=True)

full_col_list, full_row_list = np.meshgrid(np.arange(16),np.arange(16))
full_scan_list = list(zip(full_row_list.flatten(),full_col_list.flatten()))

# Make i2c_connection class object

In [ ]:
i2c_conn = helpers.i2c_connection(port, chip_addresses, ws_addresses, chip_names)

In [ ]:
def print_regs(reg_list):
    for chip_address in chip_addresses:
        chip = i2c_conn.get_chip_i2c_connection(chip_address)
        chip.read_all_block("ETROC2", "Peripheral Config")
        for reg_name in reg_list:
            reg_value = chip.get_decoded_value("ETROC2", "Peripheral Config", reg_name)
            print(f"Chip {hex(chip_address)} {reg_name}: {reg_value}")

In [ ]:
def make_baselines(do_config = True, power_mode = 'high', calibrate_chip=True, calibrate_pixels=None, make_baselines=False, save_baselines=False):
    
    if do_config:
        log_action_v2(log_path, "DAQ", "Config", "Set peripherals and Disable all pixels")
        i2c_conn.config_chips(do_set_chip_peripherals=True, do_disable_all_pixels=True)
        for address in chip_addresses:
            if power_mode == 'low':
                log_action_v2(log_path, "DAQ", "Config", "Low Power")
            elif power_mode == 'high':
                log_action_v2(log_path, "DAQ", "Config", "High Power")
            else:
                log_action_v2(log_path, "DAQ", "Config", "Set Power")
            i2c_conn.config_power_mode(chip_address = address, scan_list = full_scan_list, power_mode = power_mode)
    
    now = datetime.datetime.now().isoformat(sep=' ', timespec='seconds')
    
    if(calibrate_chip):
        log_action_v2(log_path, "DAQ", "Config", "Baseline")
        i2c_conn.config_chips(do_auto_calibration=True)
        log_action_v2(log_path, "DAQ", "Config", "Baseline Done for all pixels")
    
    elif(calibrate_pixels is not None):
        log_action_v2(log_path, "DAQ", "Config", "Baseline")
        histfile = histdir / 'BaselineHistory.sqlite'
        for idx, address in enumerate(chip_addresses):
            bl_nw_dict = {
                'row': [],
                'col': [],
                'baseline': [],
                'noise_width': [],
                'timestamp': [],
            }
            for row,col in calibrate_pixels:
                i2c_conn.auto_cal_single_pixel(chip_address=address, row=row, col=col, bl_nw_output=bl_nw_dict)
            bl_nw_df = pandas.DataFrame(data = bl_nw_dict)
            bl_nw_df['chip_name'] = chip_names[idx]
            with sqlite3.connect(histfile) as sqlconn:
                bl_nw_df.to_sql('baselines', sqlconn, if_exists='append', index=False)

            bl_nw_df = bl_nw_df.groupby(['row', 'col']).last().reset_index()
            df = i2c_conn.BL_df[address]
            # Set index for updating
            df.set_index(['row', 'col'], inplace=True)
            bl_nw_df.set_index(['row', 'col'], inplace=True)
            # Update df with tmp_df
            df.update(bl_nw_df)
            # Reset index to original format
            df.reset_index(inplace=True)
            i2c_conn.BL_df[address] = df
        log_action_v2(log_path, "DAQ", "Config", "Baseline Done for selected pixels")
    
    if(make_baselines):
        for idx, chip_address in enumerate(i2c_conn.chip_addresses):
            current_df = i2c_conn.BL_df[chip_address]
            pivot_df = current_df.pivot(index=['row'], columns=['col'], values=['baseline', 'noise_width'])
            i2c_conn.make_BL_NW_2D_maps(pivot_df, i2c_conn.chip_names[idx], "SEU Testing", fig_outdir, now)
            i2c_conn.make_BL_NW_1D_hists(current_df, i2c_conn.chip_names[idx], "SEU Testing", fig_outdir, now)
    if(save_baselines):
        i2c_conn.save_baselines(hist_dir=histdir, save_notes=now)
    
    time.sleep(0.5)

# Config chips

#### Key is (WS Init, All Offsets 20)

In [ ]:
log_action_v2(log_path, "DAQ", "Config", "PixelID, Set peripherals and Disable all pixels")
i2c_conn.config_chips(
    do_pixel_check=True,
    do_set_chip_peripherals=True,
    do_disable_all_pixels=True
)

In [ ]:
log_action_v2(log_path, "DAQ", "Config", "Baseline")
i2c_conn.config_chips(
    do_auto_calibration=True
)
log_action_v2(log_path, "DAQ", "Config", "Baseline Done for all pixels")

In [ ]:
make_baselines(do_config = False, power_mode = 'high', calibrate_chip=False, calibrate_pixels=None, make_baselines=False, save_baselines=True)

In [ ]:
# log_action_v2(log_path, "DAQ", "Config", "Initial Only")
# i2c_conn.config_chips(
#     do_set_chip_peripherals=True
# )

In [ ]:
reg_list = [
    "PLL_FBDiv_clkTreeDisable",
    "PLL_FBDiv_skip",
    "PLL_BiasGen_CONFIG",
    "PLL_CONFIG_I_PLL",
    "PLL_CONFIG_P_PLL",
    "PLL_R_CONFIG",
    "PLL_vcoDAC",
    "PLL_vcoRailMode",
    "PLL_ENABLEPLL"
]
print_regs(reg_list=reg_list)

## Dump I2C Config

In [ ]:
for chip_address,ws_address,chip_name in zip(chip_addresses, ws_addresses, chip_names):
    i2c_conn.i2c_dumping(chip_address=chip_address, ws_address=ws_address,outdir=config_outdir, chip_name=chip_name, fname='FNAL', full=True)

### Calibrate PLL

In [ ]:
log_action_v2(log_path, "DAQ", "Config", "Calibrate PLL")
i2c_conn.calibratePLL(chip_addresses[0], chip=None)
i2c_conn.calibratePLL(chip_addresses[1], chip=None)
i2c_conn.calibratePLL(chip_addresses[2], chip=None)
i2c_conn.calibratePLL(chip_addresses[3], chip=None)

### Reset Global Readout

In [ ]:
log_action_v2(log_path, "DAQ", "Config", "Reset Global Readout")
i2c_conn.asyResetGlobalReadout(chip_addresses[0], chip=None)
i2c_conn.asyResetGlobalReadout(chip_addresses[1], chip=None)
i2c_conn.asyResetGlobalReadout(chip_addresses[2], chip=None)
i2c_conn.asyResetGlobalReadout(chip_addresses[3], chip=None)

### Calibrate FC for all I2C

In [ ]:
log_action_v2(log_path, "DAQ", "Config", "Calibrate Fast Command")
i2c_conn.asyAlignFastcommand(chip_addresses[0], chip=None)
i2c_conn.asyAlignFastcommand(chip_addresses[1], chip=None)
i2c_conn.asyAlignFastcommand(chip_addresses[2], chip=None)
i2c_conn.asyAlignFastcommand(chip_addresses[3], chip=None)

# Define Utility Functions

In [ ]:
def do_pixel_operations(scan_list, board_offsets=None, noisy_pixels=None, power_mode='high', Bypass_THCal=True, QInjEn=True):
    log_action_v2(log_path, "DAQ", "Config", "Pixel Operations")
    i2c_conn.enable_select_pixels_in_chips(scan_list, Qsel=30, QInjEn=QInjEn, Bypass_THCal=Bypass_THCal, power_mode=power_mode, verbose=False)

    if board_offsets is not None:
        time.sleep(0.5)
        
        for chip_address in chip_addresses[:]:
            if chip_address not in board_offsets:
                continue
            
            chip = i2c_conn.get_chip_i2c_connection(chip_address=chip_address)

            if(Bypass_THCal):
                i2c_conn.set_chip_offsets(chip_address, pixel_list=scan_list, offset=board_offsets[chip_address], chip=chip, verbose=True)
            else:
                for row, col in scan_list:
                    print(f"Operating on chip {hex(chip_address)} Pixel ({row},{col}) and setting TH_offset to {board_offsets[chip_address]}")
                    chip.row = row
                    chip.col = col
                    chip.read_decoded_value("ETROC2", "Pixel Config", 'TH_offset')
                    chip.set_decoded_value("ETROC2", "Pixel Config", 'TH_offset', board_offsets[chip_address])
                    chip.write_decoded_value("ETROC2", "Pixel Config", 'TH_offset')

    if noisy_pixels is not None:
        time.sleep(0.5)
        
        for chip_address in chip_addresses:
            
            if chip_address not in noisy_pixels:
                continue
            
            chip = i2c_conn.get_chip_i2c_connection(chip_address)
            
            for row, col in noisy_pixels[chip_address]:
                print(f"Masking from trigger of chip {hex(chip_address)} Pixel ({row},{col})")
                chip.row = row
                chip.col = col

                chip.read_all_block("ETROC2", "Pixel Config")

                chip.set_decoded_value("ETROC2", "Pixel Config", 'DAC', 1023)
                chip.set_decoded_value("ETROC2", "Pixel Config", 'disTrigPath', 1)
                chip.set_decoded_value("ETROC2", "Pixel Config", 'Bypass_THCal', 1)

                chip.write_all_block("ETROC2", "Pixel Config")

    time.sleep(0.5)
    

In [ ]:
def close_pixels_by_broadcast(Bypass_THCal=True):
    log_action_v2(log_path, "DAQ", "Config", "All Pixel Operations")
    
    for chip_address in chip_addresses[:]:
        chip = i2c_conn.get_chip_i2c_connection(chip_address)
        chip.row = 0
        chip.col = 0
        chip.read_all_block("ETROC2", "Pixel Config")

        chip.set_decoded_value("ETROC2", "Pixel Config", 'TH_offset', 63)
        chip.set_decoded_value("ETROC2", "Pixel Config", 'Bypass_THCal', 1 if Bypass_THCal else 0)
        chip.set_decoded_value("ETROC2", "Pixel Config", 'DAC', 1023)

        chip.broadcast = True
        chip.write_all_block("ETROC2", "Pixel Config")
        chip.broadcast = False
        
        print(f"{Bypass_THCal} for Bypass_THCal, and Set TH_Offset to 0x3f and DAC to 0x3ff for all pixels of chip: {hex(chip_address)}")

In [ ]:
def run_daq(run_number, fccounter_time=7, config_time=31, save_to_file=False):
    histfile = histdir / 'InvalidFCCounterHistory.sqlite'
    fc_start_time = time.time()
    cg_start_time = time.time()
    while True:
        if time.time()-fc_start_time>=fccounter_time:
            log_action_v2(log_path, "DAQ", "Config", "invalidFCCount Read")
            now = datetime.datetime.now().isoformat(sep=' ', timespec='seconds')
            ifcc_dict = {
                'invalid_fc_counter': [],
                'timestamp': [],
                'run_number': [],
                'chip_name': [],
            }
            for idx,chip_address in enumerate(chip_addresses):
                chip = i2c_conn.get_chip_i2c_connection(chip_address)
                # chip.read_all_block("ETROC2", "Peripheral Status")
                chip.read_decoded_value("ETROC2", "Peripheral Status", 'invalidFCCount')
                value_invalidFCCount = chip.get_decoded_value("ETROC2", "Peripheral Status", "invalidFCCount")
                print(f"Chip {hex(chip_address)} Invalid FC Counter: {value_invalidFCCount}")
                ifcc_dict['invalid_fc_counter'].append(value_invalidFCCount)
                ifcc_dict['timestamp'].append(now)
                ifcc_dict['run_number'].append(run_number)
                ifcc_dict['chip_name'].append(chip_names[idx])
            if(save_to_file):
                ifcc_dict = pandas.DataFrame(data = ifcc_dict)
                with sqlite3.connect(histfile) as sqlconn:
                    ifcc_dict.to_sql('baselines', sqlconn, if_exists='append', index=False)
            fc_start_time = time.time()
            
        if time.time()-cg_start_time>=config_time:
            print("Reading chip config...")
            for chip_idx in range(len(chip_names)):
                now = datetime.datetime.now().isoformat(sep='T', timespec='seconds')
                i2c_conn.i2c_dumping(chip_address=chip_addresses[chip_idx], ws_address=ws_addresses[chip_idx], outdir=config_outdir, chip_name=chip_names[chip_idx], fname=f'DuringRun_{run_number}_{now}', full=False)
            cg_start_time = time.time()
            fc_start_time = time.time()

# Run DAQ (Actual SEU Data Taking Starts Here)

In [ ]:
col_list = [8, 2, 2]
row_list = [0, 0, 2]
scan_list = list(zip(row_list, col_list))

board_offsets = {
    0x60: 0x0f,
    0x61: 0x0f,
    0x62: 0x0f,
    0x63: 0x0f,
}
noisy_pixels = None
power_mode = 'high'

In [ ]:
log_action_v2(log_path, "DAQ", "Config", "Load I2C Config")
for chip_idx in range(len(chip_names)):
    i2c_conn.i2c_loading(chip_address=chip_addresses[chip_idx], ws_address=ws_addresses[chip_idx], outdir=config_outdir, chip_name=chip_names[chip_idx], fname='FNAL', full=True)
reg_list = [
    "EFuse_Prog",
    "singlePort",
    "serRateLeft",
    "serRateRight",
    "onChipL1AConf",
    "chargeInjectionDelay",
    "triggerGranularity",
    "fcClkDelayEn",
    "fcDataDelayEn",
    "PLL_ENABLEPLL"
]
print_regs(reg_list=reg_list)

In [ ]:
# make_baselines(do_config=False, power_mode=power_mode, calibrate_chip=True, calibrate_pixels=None, make_baselines=False, save_baselines=True)
make_baselines(do_config=False, power_mode=power_mode, calibrate_chip=False, calibrate_pixels=scan_list, make_baselines=False)

In [ ]:
close_pixels_by_broadcast(Bypass_THCal=False)

In [ ]:
do_pixel_operations(scan_list, board_offsets = board_offsets, noisy_pixels = noisy_pixels, power_mode = power_mode, Bypass_THCal=False, QInjEn=True)

# Running

In [ ]:
current_run_name = "Run_0"

In [ ]:
for chip_idx in range(len(chip_names)):
    now = datetime.datetime.now().isoformat(sep='T', timespec='seconds')
    i2c_conn.i2c_dumping(chip_address=chip_addresses[chip_idx], ws_address=ws_addresses[chip_idx],
                     outdir=config_outdir, chip_name=chip_names[chip_idx], fname=f'PreRun_{current_run_name}_{now}', full=False)

In [ ]:
log_action_v2(log_path, "DAQ", "Config", "Start Prerun InvalidFCCounter Dumping")
run_daq(current_run_name, fccounter_time=3, config_time=6001, save_to_file=False)

In [ ]:
print(datetime.datetime.now().isoformat(sep='T', timespec='seconds'))
log_action_v2(log_path, "DAQ", "Config", "Start SEU Beam")
run_daq(current_run_name, fccounter_time=28, config_time=60, save_to_file=True)

In [ ]:
print(datetime.datetime.now().isoformat(sep='T', timespec='seconds'))
log_action_v2(log_path, "DAQ", "Config", "Stop SEU Beam")
for chip_idx in range(len(chip_names)):
    now = datetime.datetime.now().isoformat(sep='T', timespec='seconds')
    i2c_conn.i2c_dumping(chip_address=chip_addresses[chip_idx], ws_address=ws_addresses[chip_idx],
                     outdir=config_outdir, chip_name=chip_names[chip_idx], fname=f'PostRun_{current_run_name}_{now}', full=False)